# Vytváření vlastních tříd

Uživatel Pythonu není omezen jen na užívání tříd, které nabízí standardní 
knihovny jazyka resp. knihovny třetích stran. Vlastní podstata objektově
orientovaného programování spočívá ve vytváření vlastních tříd, které modelují
nějakou množinu objektů z reálného světa či ze světů vědecko-technických abstrakcí.

## Vnější pohled na třídu

Jak jste již poznali, objekty jsou navenek charakterizovány svým chováním tj. tím
jaké metody můžete na daném objektu volat (počítaje v to i obecné vestavěné funkce
a operace). Zopakujme si to na příkladu objektu seznamu.

In [150]:
seznam = [1, 5, 0] # vytvoření seznamu

seznam.sort() # volání metody
print(str(seznam)) # volání obecné vestavěné funkce 'str'

print(seznam * 5) # volání operace násobení nad objektem 

[0, 1, 5]
[0, 1, 5, 0, 1, 5, 0, 1, 5, 0, 1, 5, 0, 1, 5]


Už pouhým pohledem na tento ukázkový kód zjistíme, že náš seznam je tříditelný (má
smysl nad ním volat metodu `sort`), je možné vypsat jeho textovou representaci
(tj. je převoditelný na řetězec) a je možné ho násobit celým číslem.

To vše samozřejmě platí i pro všechny ostatní objekty dané třídy. Navíc při použití
daných metod (funkcí, operací) získáme obdobné výsledky 
(liší se jen různými konkrétními položkami jednotlivých seznamů).

Obecně proto platí, že třída je jednoznačně popsaná tím, že stanovíme metody, 
které lze volat na její instance (včetně operací a volání vestavěných funkcí),
a určíme jaký efekt budou mít tyto metody pro objekt v určitém stavu, jinak 
(a stručněji) řečeno stanovíme **rozhraní** objektů.

## Vnitřní pohled na třídu

Aby mohly objekty třídy nabízet určitou funčnost musí nějak interně representovat 
svůj stav. U objektů uživatelských tříd je stav representován vnitřními 
(pod)objekty. Jinak řečeno každý objekt uživatelské třídy je složen z alespoň
jednoho (pod)objektu jiné třídy. Tyto podobjekty je nutno v rámci daného podobjektu
nějak jednoznačně identifikovat, k čemuž slouží tzv. **atributy**.

Atribut je obdobou proměnné, která je však omezená jen na na jediný objekt.
Stejně jako proměnná je to jen štítek, který (dočasně) identifikuje (pod)objekt.

Podobjekty mohou být nejen instance jednoduchých tříd jako jsou čísla nebo řetězce,
ale mohou to být i například i kontejnery či dokonce objekty dalších uživatelských
tříd.

Nejdříve vytvoříme 
triviáalní  model kasičky (prasátka),
do něhož můžeme vkládat libovolné peněžní prostředky (není to přiznávám příliš užitečné, ale nějak se začít musí). 


Pro jednoduchost budeme předpokládat, že kasička je bezedná 
tj. lze do ní vložit libovolné množství peněz 
(obecně je velmi složité zjistit, kolik prostoru mohou zaujímat
mince v prostoru kasičky).

Dále budeme předpokládat následující chování:
*  kasičky nelze vyjímat peníze, lze ji jen rozbít a získat tak najednou celou sumu
*  po rozbytí nelze kasičku už použít (nelze do ní vkládat peníze)

Z modelu je navíc zřejmé, že každý objekt kasičky (může existovat libovolné množství objektů kasiček) by měl podporovat dvě metody (modelující možné interakce s kasičkou):

1) metoda *pridej_penize* 

Tato metoda přijímá jako parametr obnos, což je kladné celé číslo (naše kasička nebude implementovat přidání zlomků základní peněžní jednotky, pro koruny je to zbytečné). Přidáná suma není v našem modelu shora omezená (máme bezednou kasičku a koho by nepotěšilo například například přidání bilión korun).

Pokud bude vše v pořádku, metoda nemusí nic vracet (jen změní stav kasičky). Je však jasné, že může dojít k problémům. Za prvé se někdo může pokusit přidat zápornou částku (a pokusit se tak vytáhnout peníze z kasičky bez jejího rozbití). V realitě to nejde, ale v Pythonu nezabráníme použití záporné hodnoty v parametru.

Dalším typem problematického přidání je vkládání peněz do rozbité kasičky. To je podle našeho modelu nepřípustné (a ani v realitě to není snadné).

Protože při běžném použití kasičky by k těmto problematickým voláním mělo docházet jen zřídka, budeme tito situace řešit výjimkou (výjimka by měla být reakce na výjimečnou situaci). Rozhodně nemůžeme tyto situace v ignorovat a to ani v počáteční fázi, neboť uživatelé by začali kasičku využívat "netradičně", a pak by byli překvapeni, že v nové verzi jim to nefunguje (uživatelem můžete být samozřejmě i autor třídy, obecně jsou to však jiní lidé).

Ve skutečnosti jsme však nevyřešili všechny možné nedefinované či okrajové příklady. Co například přidání nulové částky? Je to trochu netradiční, ale můžeme to povolit (nemění to stav kasičky). Jak se postavíme k předání jiného než celočíselného (`int`) objektu. Můžeme se snažit přidat cokoliv:  číslo v pohyblivé řádové čárce (to by nemusel být takový problém), komplexní číslo (kasička s imaginární jednotkou korun?), řetězce, seznamy, jiné kasičky (ty mohou obsahovat jiné kasičky, takže můžeme dostat celý řetězec kasiček, navíc do kasičky lze vložit odkaz na sebe sama, to už je hluboký filozofický problém). I když běžné kasičky nejsou tak striktní (lze do nich vkládat i jiné věci než peníze), my všechny neceločíselné objekty zakážeme (při pokusu o přidání vyvoláme výjimku).

2) metoda *rozbij*

Tato metoda je mnohem jednodušší. Pokud není kasička rozbitá, pak ji rozbije (= změní její stav) a vrátí celkovou naspořenou částku (může to být i nula). Pokud je již rozbitá, pak vyvoláme výjimku (nemůžeme získat peníze z již rozbité kasičky!). Žádná další možnost již není (metodě nic nepředáváme, takže její výsledek je určen pouze stavem objektu)

Nyní již máme vše připraveno k implementaci:

In [151]:
class Kasicka: # hlavička třídy
    def __init__(self):  # konstruktor
        self.castka = 0  # (počáteční) nastavení atributu
        self.rozbita = False # (počáteční) nastavení atributu
    def pridej_penize(self, castka):  # metoda
        if not isinstance(castka, int):
            raise Exception("Částka není celé číslo")
        if castka < 0:
            raise Exception("Částka je záporná")
        if self.rozbita:
            raise Exception("Kasička je rozbitá")
        self.castka += castka  # vlastní kód metody: zvýšení uložené částky o předanou částku
    def rozbij(self):
        if self.rozbita:
            raise Exception("Kasička je rozbitá")
        self.rozbita = True
        return self.castka # a nezapomeneme vrátit konečný stav peněz

Implementace třídy začíná hlavičkou, která po kličovém slově `class` uvádí jméno třídy. V Pythonu neexistuje zcela jednotný úzus ohledně identifikátorů třídy. Klíčový dokument [PEP 8 -- Style Guide for Python Code](https://www.python.org/dev/peps/pep-0008/#class-names) doporučuje jména začínající velkým písmenem, bez použití podtržítek na  místě mezer (nová slova začínají velkým písmenem). Tuto konvenci budeme dodržovat. 

Jména vestavěných tříd (např. `int`, `str`) však tuto konvenci nedodržují, neboť jejich jména jsou úzce spojena se stejnojmenými vestavěnými funkcemi. Podobný zápis se však využívá i u tříd standardní knihovny (například `datetime.datetime`), kde je využíván pravděpodobně z historických důvodů.

Hlavička třídy končí dvojtečkou, takže je jasné, že bude následovat odsazený blok (tzv. tělo třídy). Ten obsahuje definice metod, což jsou ve skutečnosti funkce volané nad objekty dané třídy.

První z metod má podivný název `__init__` (dvě podtržítka na začátku a dvě na konci!). Metody začínající a končící dvěma podtržítky (v Pythonské slangu speciální, magické respektive *dunder* metody) mají v Pythonu pevně definovaný význam a téměř nikdy se nevolají přímo tj. zápisem `objekt.__method__()`.

Metoda označená `__init` je tzv. **konstruktor**. Tato metoda se volá při vytváření/konstrukci objektu a její funkcí je vytvořit atributy objektu (definující jeho stav) a jejich inicializace tj. nastavení na počáteční hodnotu.

Prvním parametrem konstruktoru je nově vytvořený, ale prozatím ještě prázdný a neinicializovaný objekt. Tento paarmetr se v Pythonu vždy označuje jménem `self` (vždy odkazuje na objekt nad kterým je metoda volána, resp. k níž patří tj. jakoby na sebe sama). 

Poznámka: Použití jména `self` není vynucováno překladačem (formálně může být použito jakékoliv jméno bez vypsání chyby). Je to však tak silná konvence, že její narušení se chápe jako závažná stylistická chyba. Je to podobné použití hovorového či dokonce vulgárního tvaru ve formálním dokumentu. Neovlivňuje to sice jeho čitelnost, může však vést k jeho odmítnutí komunitou (zde tedy ostatními programátory).

Náš konstruktor nemá žádné další parametry, neboť bude vytvářet objekty s identickým obsahem. Náš nový objekt kasičky (dočasně označený proměnou/parametrem `self`) bude obsahovat dva atributy (atribut je něco jako interní proměnná spojená s daným objektem). Nejdříve nastavíme atribut částka (`self.castka`) tak, aby označoval objekt nula (třídy `int`), neboť kasička je na začátku prázdná (= obsahuje O jednotek měny).

Druhý atribut `self.rozbita` určuje zda je kasička rozbitá (= `True`) nebo nikoliv (= `False`). Je zřejmé, že nové kasičky se nevyrábějí rozbité (tj. atribut má na počátku hodnotu `False`). 

Tím máme objekt plně inicializovaný. Vytváření objektu můžeme hned vyzkoušet. Objekt třídy se vytvoří použitím jména třídy jako funkce (s případnými parametry v kulatých závorkách). 

In [152]:
prasatko = Kasicka() # vznikne nový objekt odkazovaný proměnnou `prasatko`

Pro kontrolu lze vypsat hodnoty atributů nového objektu.

In [153]:
print(prasatko.castka)
print(prasatko.rozbita)

0
False


Další metodou je metoda přidávající peníze do prasátka (`pridej_penize`). I tato metoda stejně jako všechny metody nad objektem musí mít první první parametr `self`. Parametr označuje objekt, nad nímž se metoda volá (při volání je vlevo od tečky). V tomto případě však má i další parametr (`castka`), který se při volání předává běžným způsobem tj. v seznamu parametrů.

Metoda může ve svém, těle využívat libovolné atributy a metody objektu. Může samozřejmě měnit i hodnoty atributů (mohou začít odkazovat na nové objekty či se změní odkazované objekty). V našem případě zvýšíme hodnotu atributu `self.castka` o číselnou hodnotu, jež je označena parametrem `castka`.

I když má atribut stejné jméno jako parametr, jedná se o dvě různé proměnné (resp. označení). Zatímco parametr `castka` zanikne ihned pro dokončení volání metody (je to lokální proměnná), atribut existuje tak dlouho jako objekt, k němuž patří (tj. volání metody určitě přežije).

Podívejme se na obrázek, který zobrazuje na co odkazují proměnné a atributy na začátku těla metody `pridej_penize` při jejím volání na prázdný objekt (odkazovaný globální proměnou `prasatko`) s přidávanou částkou 10.

In [154]:
prasatko.pridej_penize(10)

![Proměnné, parametry a atributy při volání metody](refinmethod.png)

Jak lze vidět, objekt třídy odkazován dvěma proměnnými: globální proměnnou `prasatko` (platí v celém Jupyter notebooku) a prvním parametrem `self`. Uvnitř metody však používáme jen lokální proměnné a parametry, nebo uvnitř metody nevíme jaké existují globální proměnné resp. jaké objekty právě označují).  Stačí vědět, že parametr `self` označuje objekt, s nímž máme pracovat. Objekt samotný odkazuje pomocí svých atributů na další dva podobjekty (ty jsou primárně dostupné jen přes svůj rodičovský objekt). Všimněte si, že výraz `castka` (použije se parametr `castka`) odkazuje jiný objekt, než výraz `self.castka`.

> **Úkol**: Jak se obrázek změní po přidání/přičtení částky (tj. na konci metody)?

Atribut `self` částka bude ukazovat na objekt `int:10`. Ten může být sdílen s parametrem `castka` nebo se může jednat o další kopii. Oba přístupy jsou identické, neboť objekty čísel nelze měnit. Jak bylo řečeno dříve, u malých čísel Python preferuje sdílení. 

Nový graf proměnných a objektů lze vidět na následujícím obrázku (objekt int:0 je šedý, neboť jej již nikdo neoznačuje a je tak de iure neexistující).

![Proměnné, parametry a atributy při volání metody, stav 2](refinmethod2.png)

> **Úkol:**: Jak vypadá systém proměnných po dokončení volání funkce (po návratu do globálního kontextu)?

Po návratu zanikne lokální kontext a a všechny jeho (lokální) proměnné (`self` a `castka`). Objekt (a jeho podobjekty) však nezaniknou, neboť jsou odkazovány (označeny) proměnnou `prasatko`. Stav ukazuje poslední obrázek.

![Proměnné, parametry a atributy při volání metody, stav 3](refinmethod3.png)

Vraťme se ještě k metodě `pridej_penize`, která ještě před vlastní akci zvýšení uložené částky kontroluje přípustnost daného volání.

Nejdříve kontroluje, zda je předaný objekt třídy `int`. K tomu využívá vestavěnou metodu `isinstance`. Ta očekává dva parametry: objekt a třídu. Vrací `True` je-li objekt instancí dané třídy.

In [155]:
isinstance(2, int)

True

In [156]:
isinstance("Eldar", str)

True

In [157]:
isinstance(prasatko, Kasicka) # je prasátko kasičkou

True

Pokud není objekt předaný jako parametr třídy `int`, pak je vyvolána výjimka pomocí příkazu `raise`. Příkaz `raise` přeruší vykonávání programu a pokud program na výjimku nezareaguje (to zatím ještě neumíme) pak je i ukončen. Argumentem je nově vytvářený objekt výjimky (zde základní třídy `Exception`).

> **Úkol**: Ověření na základě příslušnosti ke třídě `int` není optimální. Proč? Jak to napsat lépe?

Problémem je skutečnost, že celá čísla mohou být representována i pomocí objektů třídy `float` (a dokonce i dalšími jako je `complex` nebo `fraction`). Základní možností je pokusit se o přetypování na celé číslo a zpět. Pokud obdržíme původní hodnotu, pak lze číslo representovat jako `int` a je tudíž celé. Bylo by to však nutné provést pro všechny potenciální číslené typy schopné uchovávat celá čísla. V praxi by tak mohlo stačit využití vestavěné metody `is_integer` třídy *float*.

In [158]:
# testovací prográmek
x = 10.0  
# zkuste i použití zlomeku, zamyšlete se nad výsledkem a jeho použitelností
# import fractions
# x = fractions.Fraction(6,2)
if isinstance(x, int) or x.is_integer():
    print("Celé číslo")

Celé číslo


Testování dalších dvou podmínek použitelnosti metody je jednoduché (kladnost přidané částky a stav nerozbytí).

Stejně tak jednoduchá je i implementace metody pro rozbití kasičky (kontrola a následná změna jediného atributu). Nynínovou třídu vyzkoušímě (a pro jistotu si vytvoříme novou kasičku).

In [159]:
k = Kasicka() # vytvoříme
k.pridej_penize(1_000_000) # přidáme pár korun
k.pridej_penize(1)  # a ještě jednou
print(k.rozbij()) # rozbijeme a vypíšeme celkovou sumu

1000001


Musíme vyzkoušet i problematická (nepřípustná volání). 

In [164]:
k = Kasicka()
try:   # zkusíme to risknout (i když očekáváme výjimku)
    k.pridej_penize(-10)
except Exception as e: # pokud nastane pak ji zachytíme
    print(e) # a vypíšeme

Částka je záporná


Program skončil s výjimkou, ale kasička stále existuje (v notebooku existují všechny vytvořené, dokud notebook neuzavřeme)

In [165]:
k.rozbij()

0

In [166]:
k.pridej_penize(100)

Exception: Kasička je rozbitá

#### Zapouzdření

V popisu požadovaných rysů kasičky jsem kladl velký důraz na to, že dokud kasičku nerozbijeme, nelze zjistit kolik peněz obsahuje. To však není evidentně pravda:

In [ ]:
k = Kasicka() # vytvoříme novou
k.pridej_penize(100)

print(k.castka)

Nyní víme, že v kasičce je 100 korun, ale přesto jsme ji nerozbili.

Situace je však ještě horší.

In [ ]:
k.castka = 0

To je vykradení kasičky za bílého dne! Může však být ještě hůře: 

In [ ]:
k.castka = -100

Nyní máme kasičku zadluženku (tj. kasička do níž lze vkládat např. služní úpisy). Podobně můžeme z rozbité kasičky udělat nerozbitou (je to lepší než oprava, nedokážeme totiž rozlišit zda je skutečně nerozbytá nebo znovuskříšená i s původním obnosem).

Jak je to vůbec něco takového možné?

Důvod je jednoduchý. V Pythonu je dáno pouze dohodou, co smíme dělat (co je košer) a co nikoliv (co je hucpe).

**Obecná dohoda je taková, že můžeme volat pouze metody (ať již přímo či nepřímo). Ty musí být napsány tak, že nevznikne žádný nedefinovaný stav. Naopak atributy by neměly být vně metod dané třídy použité.**

Tato dohoda vychází ze základního principu objektově orientovaného programování. S objekty lze interagovat pouze pomocí volání metod z veřejného rozhraní. Interní (datová) representace (tj. struktura atributů) je naopak skrytá.

Tento princip se označuje jako **zapouzdření** (angl. *encapsulation*). Cílem zapouzdření není skrýt (tajná) data, ale zabránit závislosti okolního kódu na struktuře a zamezit nepřípustné modifikaci interních dat (viz dluhová kasička). 

Tento princip lze vysvětlit na náramkových hodinkách. Veřejným rozhraním je ciferník, je standardizovaný a běžně se nemění.  Vnitřní mechanismus je skrytý. Pokud by byl snadno viditelný pak by vnější pozorovatel mohl obejít ciferník a využívat pouze vnitřní informace (např. pozice ozubených koleček). Tato (nechtěná) závislost by vedla ke zbytečně složitému algoritmu přístupu (kolečka nejsou primárně určena pro zobrazení času), neautorizovaným změnám (není překvapivé, že po vyjmutí pár koleček nemusí hodinky fungovat a navíc asi přijdeme i o záruku) avšak především by téměř znemožnila upgrade interního mechanismus hodinek (poN přehodu na digitální technologii, již nelze kolečka pro čtení hodinek využívat, naopak ciferník může zůstat beze změny).

Některé jazyky si zapouzdřenost vynucují automaticky, jiné mají nástroje jak si zapouzdřenost vynutit explicitně pro danou třídu, Python předpokládá, že programátoři jsou dospělí a tak vědí co činí (resp. co by činit neměli).

V některých případech však základní rozdělení na veřejné metody a skryté atributy nestačí. Relativně často se používají pomocné metody, které nejsou součástí veřejného rozhraní, ale používají se výhradně v ostatních metodách (v zásadě je to dodání další skryté vrstvy schované ještě pod veřejným rozhraním).

Tyto metody lze vyznačit tím, že se na začátku jejich identifikátoru použije jedno podtržítko. V tomto případě pomůže trochu i Python, který trochu zkomplikuje jejich volání z vnějšku (zabrání se tak nechtěnému použití).
Signifikantnější je však neuvedení popisu metody ve veřejné dokumentaci (co není popsáno, jako by nebylo).

Na druhou stranu některé objekty nabízejí veřejné atributy. Zde je situace jasná. Atribut je veřejný jen tehdy, když je explicitně zmíněn v dokumentaci. Dokumentace také určuje, zda je pouze pro čtení (typičtější případ) nebo i pro zápis. 

### Příklad: Matematický model

Na začátek vytvoříme třídu, jejíž instance budou representovat velmi jednoduché
matematické objekty: uzavřené intervaly na množině reálných čísel.

> Poznámka: V rámci programátorské praxe budete jen výjimečně vytvářet třídy
> modelující matematické pojmy (programování není matematika). Matematické 
> pojmy však mají pro výuku základů programování 
> dvě hlavní výhody: jsou jednoznačně definované (= všichni mají stejnou představu
> o příslušných objektech).
> a mohou být extrémně jednoduché (což pro modely reálných objektů naplatí).

Nejdříve si určíme jaké metody budou objekty nabízet (tím určíme jejich rozhraní
a nedefinujeme jejich chování). 

U takto jednoduchých tříd je nejjednodušší začít návrhem malého programu, který 
bude využívat objekty naší nové třídy (program je prozatím nefunkční, neboť Python
neposkytuje třídu `Interval`).

```python
interval1 = Interval(0.0, 1.0)  # vytvoření intervalu konstruktorem
interval2 = Interval(0.5, 3.0) #  vytvoření intervalu konstruktorem

print(str(interval1)) # objekt lze převést na text -> "<0, 1>"
print(interval2.length()) # je možno zjistit délku intervalu -> 2.5

prunik = interval1.intersection(interval2) # vrátí průnik jako nový interval
print(prunik) # -> <0.5, 1>

print(2.0 in prunik) # použití operátoru `in` -> False
```

Podívejme se detailněji na jednotlivé požadované metody.

Při vytváření nových objektů se volá speciální metoda, která se označuje jako 
**konstruktoru**.  Tato metoda přijímá parametry, pomocí 
nichž musí tzv. inicializovat objekt. To znamená že musí:

1. vytvořit podobjekty, které budou popisovat stav objektu dané třídy
2. vytvořit atributy, které budou podobjekty odkazovat (a tak je budou v rámci
    objektu identifikovat)

V našem případě má konstruktor dva parametry, které určují dolní a horní mez
intervalu (v tomto pořadí). 

Otázkou je, co by měl konstruktor dělat v případě, že programátor zadá meze
v opačném pořadí tj. např.  `Interval(2,1)`.

Nabízí se tři základní možnosti:

1. ignorovat problém a vytvořit objekt, jehož meze jsou nesprávně nastaveny
2. prohodit meze tak, aby byly správně uspořádané (bez upozornění)
3. prohodit meze tak, aby byly správně a vypsat upozornění
4. odmítnout neplatné meze a vyhodit výjimku (což typicky povede k ukončení 
   programu).
   
Je zřejmé, že první (pštrosí) přístup není akceptovatelný. Vždy platí, že objekt
by měl být po svým vytvoření v konzistentním stavu. V opačném případě jen
oddalujeme problém. Pokud budou meze nesprávně nastaveny, pak nás nesmí překvapit,
že délka intervalu bude záporná a funkční nebude ani interval pro hledání průniku
(oba výpočty, lze samozřejmě upravit, tak aby fungovaly i s neplatnými mezemi,
ale bude to přinášet hodně práce navíc).

Druhý přístup se jeví jako perspektivnější a navíc jako vstřícný k uživatelům.
Obecně však platí, že byste se měli vyhnout "magickému" chování, a to především
v případě, kdy slouží k zakrývání chyb či nekonzistencí. 
Můžete tak totiž nechtěně bránit včasné detekci chyb. Ty se tak mohou projevit až
později a bohužel často s mnohem ničivějším účinkem.

Doporučit tak lze jen poslední dva přístupy. Tolerance s upozorněním však není
tak široce podporována jako mechanismus výjimek (obecně není zřejmé, jak bude
upozornění vypisováno). Proto pokud máte svobodu volby, využívejte mechanismu
výjimek i za cenu, předčasného ukončení programu.

Na objekt intervalu bude možno volat obecnou vestavěnou funkci `str`. Tato
funkce by měla vrátit textovou representaci intervalu v podobě objektu řetězce.

Tato funkce by měla vrátit representaci určenou primárně pro člověka, neboť
se využívá primárně pro generování textových výstupů a pro ladění. Obecně
nemusí být převoditelná zpět na objekt (tj. nemusí obsahovat všechny informace).
V našem případě existuje standardní notace ve tvaru `<D, Y>`, kde `D` je dolní
mez a `H` mez horní.

Interval bude navíc podporovat i vestavěnou funkci `len`, která by měla (již podle
svého názvu) vracet délku intervalu tj. `H` – `D`.

Pomocí standardního zápisu volání metody nad objektem je implementována operace,
která vrací průnik dvou intervalů (nad prvním je metoda volána, druhý je
předán jako parametr). Výsledkem je nový objekt representující výsledný interval.
Při návrhu této metody, je nutno zohlednit fakt, že dva intervaly mohou mít
prázdný průnik. Jak se v tomto případě metoda zachová?

Nemůžeme vyhodit výjimku, neboť prázdný průnik je běžným výsledkem a nejedná
se tudíž o výjimečný a tím méně chybový stav. 

Dalším řešením je zavést objekt,
který representuje prázdný interval. To však přináší další obtíže:

1. objekt prázdného intervalu není možno vytvořit pomocí konstruktoru 
(je-li `H` >= `D`, pak interval obsahuje alespoň jeden bod a není tudíž prázdný) 
2. Jak prázdný interval převedeme na řetězec? Zápis $\emptyset$ je sice možný,
ale vztahuje se spíše na množiny a nikoliv na intervaly (i když interval je
samozřejmě množina)
3. Jaká je délka prázdného intervalu? Nemůže to být nula, neboť to je délka
jednoprvkového intervalu tj. intervalu, jehož horní mez = dolní mez.

Bohužel v tomto případě není možné najít zcela uspokojivé a přitom jednoduché 
řešení. Pro naše účely zvolíme následující přístup:

1. prázdný interval bude vytvářen voláním konstruktoru bez parametrů 
tj. `Interval()`. Navíc umožníme i jednoparametrický konstruktor (vytvářející
jednoprvkové intervaly). To sice není nezbytné usnadňuje to však použití, neboť
není nutné dvakrát opakovat totéž číslo (resp. tutéž proměnnou). 
2. řetězcovou representací prázdného intervalu bude `<>`
3. volání funkce `len` na prázdný objekt je nedefinované a proto bude vyhozena
výjimka. Pro testován, zda je interval prázdný je proto nutné implementovat
dodatečnou metodu `isEmpty()`.

Příklad použití metody `isEmpty`:
```python
if prunik.isEmpty():
    print("prazdný průnik")
```

Rozhraní už máme přesně definované a nyní tedy můžeme přejít k návrhu vnitřní
implementace. 

Základní interní intervalů representace je zřejmá: 
dva atributy, z nichž jeden bude odkazovat
reálné číslo representující dolní mez (s jménem např. `low`) a druhý 
reálné číslo representující 
horní mez (`high`). Otázkou je však representace prázdného atributu.

Zde existují dvě možnosti: jednou je přidání dalšího atributu `empty`, 
který bude nést
logickou hodnotu, signalizující prázdnost intervalu. Druhou možností je využití
nějaké neplatné kombinace dolní a horní meze (např. dolní meze větší než horní).

Každé z obou řešení má své nevýhody.
Přidání dalšího atributu a podobjektu 
zvětší velikost objektu naší třídy (v našem případě minimálně o 9 bytů na 64-bitovém
systému). Navíc, v případě prázdného intervalu jsou atributy dolní a horní meze
nevyužité (mohou obsahovat cokoliv). Alternativně lze v případě prázdného seznamu
definovat jen atribut `empty`, což však může být matoucí (především pro programátory
jazyků, v nichž nelze měnit počet atributů na základě stavu objektů). 

Použití neplatného stavu (= nepřípustné kombinace hodnot atributů) šetří paměť,
je však náchylnější k chybné interpretaci nebo k chybnému použití. Pokud je třída
využívána dlouhodobě či je vytvářena větším týmem může dojít k dezinformacím
(nepřípustný stav je chybně interpretován).

V současnosti, kdy rozsah paměti neni omezujícím faktorem **je lepší preferovat
explicitní representaci oproti (zne)užití neplatných hodnot**. Toto pravidlo však
nemá absolutní platnost.

Nyní už můžeme přistoupit k implementaci (třídu je nutné implementovat v v jediné
buňce):

In [167]:
class Interval: # hlavička třídy
    def __init__(self, low=None, high=None):
        self.empty =  low is None and high is None # prázdný interval
        self.low = low if low is not None else high
        self.high = high if high is not None else low
        
    def __str__(self):
        return f"<{self.low}, {self.high}>" if not self.empty else "<>"

    def length(self):
        if self.empty: # je-li prázdný
            raise ValueError("Undefined operation for empty interval") 
            # pak vyhoď výjimku
        return self.high - self.low
    
    def intersection(self, other):
        if self.empty or other.empty: 
            # průnikem dvou intervalů, z nichž je alespoň jeden prázdný 
            return Interval()  # je prázdný interval
        low = max(self.low, other.low)
        high = min(self.high, other.high)
        if low <= high:
            return Interval(low, high)
        else:
            return Interval()
            
    def __contains__(self, x):
        if self.empty:
            return False
        else:
            return self.low <= x and x  <= self.high

Vyzkoušejme nejdříve náš návrh v podobě programu 
(je zbytečné popisovat třídu, která nefunguje):

In [168]:
interval1 = Interval(0.0, 1.0)  # vytvoření intervalu konstruktorem
interval2 = Interval(0.5, 3.0) #  vytvoření intervalu konstruktorem

print(str(interval1)) # objekt lze převést na text -> "<0, 1>"
print(interval2.length()) # je možno zjistit délku intervalu -> 2.5

prunik = interval1.intersection(interval2) # vrátí průnik jako nový interval
print(prunik) # -> <0.5, 1>

print(2.0 in prunik) # použití operátoru `in` -> False

<0.0, 1.0>
2.5
<0.5, 1.0>
False


Definice jednotlivých metod lze ve většině případů snadno interpretovat. Pro přetypování na řetězec je nutno definovat speciální (*dunder*) metodu `__str__` (odpovídá příslušné vestavěné funkci `str`, používané ke konstrukci řetězců). Testování, zda je hodnota obsažena (= operátor `in`) se definuje pomocí speciální metody
`__contains__` (to už není tak přímočaré, jména jednotlivých *dunder* metod lze nalézt např. na stránkách [Data model](https://docs.python.org/3/reference/datamodel.html)). 

Jediná metoda, která vyžaduje trochu přemýšlení je metoda `intersection`, neboť nalezení mezí průniku není zcela přímočaré. V každém případě je nutné si na papíře nakreslit jednotlivé případy protínajících se (a neprotínajících se) intervalů. Červeně je `self` interval (na něj se metoda volá) a modře `other` (je předán jako parametr).

![různé případy průniku intervalů](intervals.svg)

Z obrázku je zřejmé, že dolní mez výsledného průniku je vždy větší z obou původních mezí tj. `max(self.low, other.low)` a horní mez menší z původních horních tj. `min(self.high, other.high)`. Pokud je výsledná dolní mez větší než horní, je průnik prázdný.

>  Poznámka: Všimněte si, že objektové programování si vynucuje dosti asymetrický pohled na realitu. Vždy je tu hlavní objekt (adresát metody) a popřípadě další méně důležité objekty, které mohou být nezbytné, ale prostě hrají jen druhé housle. Pomocné objekty jsou předávány jako parametry. 
> Někdy je tato asymetrie jen čistě umělá, jako je tomu zde (oprerace průniku je symetrická a proto nezáleží, jaký objekt je adresátem (`self`) a jaký jen parametrem (`other`). Ve většině případů je však asymetrie zřejmá.

```python
seznam.append(položka); # hlavním objekt je je seznam (jen seznam změní stav)
pokladnicka.pridej_penize(castka); # pokladnicka je opět hlavním objektem

castka.pridej_mne_do(pokladnicka); # opačné chápání je podivné (mění se jen parametr, a jméno je dlouhé)

```

> Tato asymetrie, je obdobou asymetrie *já* versus ostatní *svět*, která je vlastní lide. Proto je někdy vhodné
namísto identifikátoru `self` využívat zájméno `já` či `můj`, které navíc koresponduje se zapouzdřením (ukrývám své já před světem). Stačí se jen ztotožnit s objektem. Lze tedy například říct: "dolní mez průniku je rovna maximu *mé* dolní meze a *jeho* dolní meze."


> **Úkol:** Vyzkoušejte program i pro jiné vstupní intervaly, především 
otestujte situace, kdy je některý z intervalů prázdný resp. jednoprvkový.

In [169]:
interval1 = Interval(0.0, 1.0)  # vytvoření intervalu konstruktorem
interval2 = Interval(1.5, 3.0) #  vytvoření intervalu konstruktorem

print(str(interval1)) # objekt lze převést na text -> "<0, 1>"
print(interval2.length()) # je možno zjistit délku intervalu -> 2.5

prunik = interval1.intersection(interval2) # vrátí průnik jako nový interval
print(prunik) # -> <0.5, 1>

print(2.0 in prunik) # použití operátoru `in` -> False

try:
    print(prunik.length())  # zde musí vzniknout výjimka
except:
    print("výjimka")

<0.0, 1.0>
1.5
<>
False
výjimka


## Komplexnější třída (graf spojení mezi městy)

Typickou úlohou v geograficky orientovaných informačních systémech je hledání nejbližších cest mezi dvěma místy, s využitím mapových podkladů. To je samozřejmě dosti komplexní problém a proto, si ho zjednodušíme.

Budeme proto hledat spojení jen mezi omezeným počtem míst (řekněme třeba mezi městy), jejichž (nejkratší) přímou vzdálenost zadáme ručně (pomocí volání metody).

Nejdříve zkusíme navrhnout rozhraní objektů třídy `Place` (tato třída může být používána v reálných aplikacích, takže volíme anglické identifikátory):

* konstruktor: parametrem je jméno místa a jeho poloha (zeměpisná šířka a délka)
* metoda pro přidání přímého spojení `add_connection`, parametrem je cílové místo a vzdálenost
* metoda pro hledání délky nejkratšího spojení `distance`, parametrem je cílové místo. Metoda najde nejkratší spojení (to může procházet více místy) a vrátí jeho vzdálenost. 

Zeměpisná poloha předávaná v konstruktoru se nepoužívá pro výpočet vzdáleností (vzdálenosti jsou v našem modelu skutečné dopravní vzdálenosti, nikoliv vzdušné vzdálenosti). Použijeme ji později pro nakreslení jednoduché mapy.

Obě metody pracující se spojením očekávají jako parametr objekt třídy `Place`, representující cíl. Obě metody tak pracují se dvěma objekty třídy `Place`. Je to jednak objekt dostupný v metodě pomocí parametru `self` (adresát metody = objekt, na nějž je metoda volána) representující počátek spojení, jednak objekt předaný jako běžný parametr representující cíl spojení. 

Tj. předpokládáme-li, že proměnné `decin` a `teplice` označují objekty příslušných měst, pak bude lze využít následujících volání:

```python
decin.add_connection(teplice, 21) # 35km z Děčína do Teplic
print(decin.distance(teplice))  # vypíše 35km  
```

Navržené rozhraní má určitý nedostatek, který jste už možná zaznamenali. Pokud existuje přímé spojení z bodu A do B, pak s vysokou pravděpodobností existuje i stejně dlouhé spojení v opačném směru (většina dopravních spojení je obousměrná). Bylo by tedy užitečné, kdyby se při přidání spojení automaticky přidalo i spojení opačným směrem. Na druhou stranu výjimečně může existovat i přímé spojení jen jedním směrem, resp. obě spojení mmohou mít různou délku.

Řešením je přidání dalšího parametru `bidi` (relativně běžně používaná zkratka za *bidirectional*) k metodě `add_connection`. Pokud bude mít hodnotu `True` pak se automaticky vloží i spojení opačným směrem (při `False` nikoliv). Navíc zde můžeme použít implicitní hodnoty parametru. Pokud ji nastavíme na `True`, pak se při běžném použití nebude vůbec uvádět (a použití bude identické s výše uvedeným příkladem).  Pokud bude výjimečně potřeba zadat jednodměrné spojení, bude volání o něco delší, ale stále přehledné (hlavně použijeme-li pojmenovaný parametr).

```python
usti.add_connection(teplice, 21, bidi=False) # umělý příklad (spojení není ve skutečnosti jednosměrné)
```

Po návrhu rozhraní, musíme navrhnout i datovou representaci objektů míst. Jednoduché je to u jména a polohy místa, neboť každé místo bude míst jen jedno jméno, jednu zeměpisnou šířku a jednu zeměpisnou délku. Tj. stačí vytvořit tři atributy objektu, které budou označovat/odkazovat příslušné hodnoty (řetězec a dvě čísla třídy `double`).

Složitější je representace přímých spojení. Každé místo může mít totiž obecně neomezený počet spojení. Navíc spojení není jednoduchá hodnota, ale minimálně hodnoty dvě (identifikace cílového místa a vzdálenost, počáteční místo je dáno umístěním dané informace). Musíme proto využít nějaké kolekce.

Klíčovou kolekcí v Pythonu je seznam. Proto ověříme nejdříve jeho použitelnost.

Za prvé potřebujeme ukládat dvojice hodnot. To lze zařídit, neboť položkami seznamu mohou být (vnořené) seznamy resp. jiné kolekce. Pokud bychom takový seznam vytvářeli ručně, pak by například mohl mít tento tvar (proměnné označují příslušné objekty):

```python
spojeniZDecina = [[usti, 15], [teplice, 21], [rumburk, 43], ...] # seznam seznamů
```

### N-tice

Namísto vnořených seznamů lze využít tzv. n-tice (angl. *tuple*), což je uspořádaná kolekce optimalizovaná pro malý počet prvků, která je navíc nemodifikovatelná (po vytvoření do ní nelze přidávat prvky, či prvky zaměňovat). N-tice se na rozdíl od seznamů uzavírají do kulatých závorek (ty lze v některých případech vynechat, v následujícím zápise to však možné není, jistě poznáte proč).

```python
spojeniZDecina = [(usti, 15), (teplice, 21), (rumburk, 43), ...] # seznam dvojic
```

Tato implementace se jeví jako rozumná. Není přirozeně zcela dokonalá. Pokud například chcete v našem seznamu vyhledat vzdálenost do Rumburku, musíme postupně procházet jednotlivé prvky a hledat ten, jehož první položka je rovna hledanému místu. Teprve pak můžeme vrátit příslušnou vzdálenost. Můžeme si na však to připravit funkci (parametr `key` je hodnota kterou hledáme, v prohledávaných dvojicí musí být vždy uváděna jako první):

In [170]:
def get_value(listOfpairs, key):
    for k, v in listOfpairs: # postupně procházíme dvojice
        if k == key:
            return v
    return None # pokud nic nenajdeme vrátíme `None`

Novinkou kódu je uvedení dvojice proměnných na místě řídící proměnné cyklu `for`. Je to obdoba paralelního přiřazení. Hodnota dvojice z procházeného seznamu je přiřazena do dvojice proměnných (i kolem této dvojice je možno napsat závorky stejně jako u n-tic).

Alternativně lze dvojice ukládat pomocí slovníku (`dict`). Pro malý počet dvojic (řádově jednotky) to  však není příliš efektivní (slovník je optimalizován pro větší množiny).  

Třídu můžeme implementovat přímo v Jupyter notebooku, není to však pohodlné. Kód třídy musí být celý obsažen v jediné vstupní buňce, což je při jeho rozsahu (několik desítek řádek) nepřehledné.

Proto zkusíme třídu naprogramovat v editoru `pycharm`. Podle návodu v první kapitole vytvořte projekt `cities` (jméno projektu nehraje v Pythonu žádnou roli, slouží pouze pro organizaci skriptů v editoru). V rámci projektu vytvořte soubor `cities.py`. Jméno souboru musí mít příponu `py`. Vlastní jméno souboru (bez přípony) se používá jako jméno modulu při importování (takže je vhodné volit krátké, ale přitom dostatečně jednoznačné jméno).

Do editoru vložte nejdříve tento text (poznámky přepisovat nemusíte):

```python
class City:
    def __init__(self, name): # konstruktor
        self.name = name      # do objektu uložíme jméno města
        self.connections = [] # seznam spojení z města je zatím prázdný

    def add_connection(self, target, distance, bidi=True):
        self.connections.append((target, distance))  
        # přidáme dvojici (město, vzdálenost) do seznamu
        if bidi:  # je-li spojení obousměrné
            target.add_connection(self, distance, bidi=False) # vložíme i opačný směr
```

**Úkol**: Co se stane, pokud by se v implementaci metody `add_connections` vkládalo opačné spojení voláním metody `add_connection` s implicitní hodnotou parametru `bidi` (tj. s hodnotu `True`).

Předpokládejme, že vkládáme obousměrné spojení mezi městy v proměnných `a` a `b`, tj. použijeme volání `a.add_connection(b, 1, bidi=True)`.
Parametr `bidi` určuje, že musí být volán i opačný směr tj. uvnitř metody `add_connection` dojde k (rekurzivnímu) volání tvaru `b.add_connection(a, 1, bidi=True)`. Protože i toto volání by bylo nastaveno jako obousměrné, vedlo by opět k novému rekurzivnímu volání metody `add_connection` tentokrát opět v původním gardu tj, `a.add_connection(b, 1, bidi=True)`. I toto volání by však využilo vložení spojení v opačném směru (tj. v podobě  `b.add_connection(a, 1, bidi=True)`). Nyní by Vám již mělo být jasné, že řetězec volání je nekonečný, neboť přidání spojení na jedné straně by vedlo k přidání na straně druhé a tak dále.
Výše uvedená implementace však opačný směr přidává jako jednosměrný a řetězec je tak ukončen ihned po druhém volání.

Před konečnou implementací nám zbývá promyslet implementaci metody `distance`.  Implementace není zcela jednoduchá neboť vyžaduje alespoň základy algoritmického myšlení tj. abstrakce univerzálně použitelných postupů pro řešení matematicky definovaných problémů.

Jak najdeme nejkratší spojení do vzdáleného města v reálném životě. Pokud máme k dispozicí mapu pak většinou stačí jediný pohled a danou cestu nalezneme (pokud bereme v potaz jen hlavní klíčové silnice tj. v našem případě silnice druhé a vyšší třídy).  Můžeme se sice splést (například v hledání cesty z Ústí do Teplic nemusí být zřejmé zda jet přes Chlumec nebo přes Řehlovice), ale chyba bude jen malá (v praxi má větší vliv kvalita silnic a aktuální dopravní situace).

Tento přístup však nelze pro náš účel použít, nemáme totiž žádné informace o vzájemné pozici měst ani globální pohled na strukturu silnic (odkud, kam a přes jaká města silnice vedou). 

Zkusíme proto strategii nepříliš inteligentních, ale vytrvalých a partogeneticky se rozmnožujících mravenců. Navíc všichni tito mravenci běhají stejnou konstantní rychlostí (třeba kilometr za den).

Nejdříve položíme jediného mravence do počátečního města. Protože je jejich partogenetické rozmnožování úžasně rychlé, je pro něj záležitostí zanedbatelného okamžiku zplodit tolik nových mravenců, kolik spojení vychází z počátečního města. Nově zrození  mravenci se ihned rozbíhají po těchto spojích. Mravenec rodič (tj. defacto praotec) zůstává v počátečním městě, kde (dožívá) ve štěstí a blahobytu (rozmnožování mu bohužel sebralo dost sil).

Pokud děti a případně i vzdálenější potomci (vzdálenější ve smyslu příbuzenství) dorazí do dalšího města pak jejich chování záleží na dvou okolnostech.

Pokud tam dorazí jako první a město je cílové, pak on a jeho předci vyhráli (štafetový) běh a my známe nejkratší vzdálenost mezi počátečním a cílovým městem (je dáno časem jejich vítězného doběhu, neboť rychlosti jsou konstantní a čas na rozmnožování je zanedbatelně malý). Pokud si mravenci-předci předávali i jména měst, kterými prošli, pak známe i nejkratší cestu.

Pokud mravenec dorazí jako první a město není cílové, pak se zachová jako praotec.  Porodí tolik mravenců, kolik je spojení z daného města a ihned je na tato spojení vyšle. Sám zůstává a dožívá v naději, že některý z jeho potomků vyhraje (jistotu však samozřejmě nemá).

Nejsmutnější je situace, kdy mravenec do některého z měst nedorazí jako první (což se pozná snadno, neboť ve městě stále dožívá první pokořitel daného města). V tomto případě je totiž zřejmé, že on resp. jeho případné potomci již nemohou vyhrát (potomci prvního mravence již vyrazily dávno předtím). Nezbývá mu nic jiného než spáchat sebevraždu (samozřejmě bez jakéhokoliv rozmnožování).

Algoritmus běžně končí tím, že první mravenec dorazí do cílového města. Další sledování mravenců je pak samozřejmě zbytečné (nakonec všichni zbývající stejně spáší sebevraždu). Pokud je však cílové město z počátečního města nedosažitelné (což by se ve Střední Evropě stávat nemělo) končí algoritmus jinak (jedním z konkurenčních výhod dobrého programátora je schopnost vidět i tyto okrajové následky).

> **Úkol**: Jak končí algoritmus mravenců-průzkumníků v případě nedosažitelnosti cílového města?

Počet mravenců dále nenarůstá (nevznikají noví) a zbývající (nesmrtelní) mravenci zůstávají ve všech městech dosažitelných z počátečního (vždy jeden).

Tento algoritmus je relativně snadno pochopitelný, **nelze** jej však bohužel přímo realizovat v běžném počítači. V jednom okamžiku totiž mezi městy pobíhá větší počet mravenců, z nichž každý by pro svou programovou representaci vyžadoval nezávislý procesor, neboť jejich činnosti tj. především rozmnožování musí být zcela nezávislé na ostatních mravencích (a to i v případě, že by přesun do města či dožití by byli representovány pouze čekáním na určitý čas bez nutnosti využití procesoru). Současné počítače sice běžně obsahují více procesorů (jader). Jejich počet je však fixní a relativně malý (bylo by podivné, pokud byste byli nuceni kupovat počítač s desítkami procesorů pro nalezení nejkratší cesty v dopravním systému mezi několika městy).

Algoritmus tak musíme trochu upravit. Namísto velkého množství rozplozujících se mravenců, použijeme jen jednoho, který však musí mít schopnost teleportace a musí být gramotný.

Tento mravenec je na začátku v počátečním městě. Zjistí si všechna přímá spojení, avšak namísto aby se do některého z nich vydal si je zapíše do svého turistického pořádníku. Napíše si odkud by se měl vydat a kam by měl dorazit. Na pořádí záznamu nezáleží.

Navíc si vytvoří tabulku, v níž má u každého města poznamenanou nejmenší ověřenou vzdálenost od města startovního. Na začátku obsahuje u startovního nulu a u všech ostatních nekonečno (náš mravenec umí dokonce i ležatou osmičku). 

Nyní vše běží ve velké smyčce. Mravenec se podívá do svého turistického pořadníku a teleportuje se do místa, ze kterého vychází první přímé spojení v pořadníku. To spojení projde a tím zjistí vzdálenost. Nezapomene samozřejmě projité spojení odstranit z pořadníku.

Poté se podívá se do tabulky. Sečte minimální vzdálenost města, odkud vycházelo právě projité přímé spojení, s ujitou vzdáleností. Pokud je tato vzdálenost menší než aktuální minimální vzdálenost města, v němž se právě nachází, pak najde všechna spojení vycházející z tohoto místa a zapíše si je do pořadníku (opět dvojici odkud a kam). Navíc přepíše minimální vzdálenost aktuálního místa v tabulce. V opačném případě (součet vzdáleností je větší) nemusí v tomto kroku provádět nic.

Nyní se vrátíme znovu na počátek smyčky. Pokud je pořadník prázdný pak algoritmus končí. Jinak se mravenec teleportuje do místaze kterého vychází první přímé spojení v pořadníku. To spojení projde a tím zjistí vzdálenost. Nezapomene samozřejmě projité spojení odstranit z pořadníku.

Poté se podívá se do tabulky. Sečte minimální vzdálenost města, odkud vycházelo právě projité přímé spojení, s ujitou vzdáleností. Pokud je tato vzdálenost menší než aktuální minimální vzdálenost města, v němž se právě nachází, pak najde všechna spojení vycházející z tohoto místa a zapíše si je do pořadníku (opět dvojici odkud a kam). Navíc přepíše minimální vzdálenost aktuálního místa v tabulce. V opačném případě (součet vzdáleností je větší) nemusí v tomto kroku provádět nic.

Nyní se vrátíme znovu na počátek smyčky. Pokud je pořadník prázdný pak algoritmus končí. Jinak se mravenec teleportuje do místa, ze kterého vychází první přímé spojení v pořadníku, ....

I když to na první pohled nemusí být zřejmé cyklus nakonec skončí (mravenec si od jisté chvíle již nezapíše nové spojení do svého pořadníčku a ten se tak nakonec vyprázdní). Navíc určitě prošel i minimální cestou, Opakuje totiž všechny cesty svých vzorů, kteří používali strategii "nas mnogo" (a v případě potřeby nás může být ještě víc). Minimální vzdálenost najde v tabulce  v řádce cílového města. V nejhorším případě tam nalezne ležatou osmičku (pokud cílové město není ze startovního dosažitelné).

Algoritmus lze samozřejmě ještě zjednodušit. Pokud známe předem délku všech přímých spojení, pak je zbytečné aby se mravenec teleportoval. Vše snadno zjistí doma pomocí tužky a papíru (jen připisuje a ruší záznamy ve pořadníku a případně snižuje hodnoty v tabulce měst). To však může provádět i počítač a to pravděpodně značně rychleji. Proto můžeme eliminovat i onoho geniálního zástupce řádu blanokřídlých.

Navíc lze v tomto případě algoritmus i mírně optimalizovat, neboť jste si jistě všimli, že mravenci procházeli i spojení, u nichž bylo již v okamžiku startu jasné, že nebudou nejrychlejší (koncové město spojení bylo dosaženo již před startem průchodu spojením). Speciálním případem těchto zbytečných průchodů je návrat do místa, z něhož vyšel předek (resp. předci). Tyto případy lze za pomoci tabulky aktuálních minimálních vzdáleností snadno eliminovat, neboť je nemusíme zařazovat do pořadníku.

Dalším vylepšením je eliminace počátečního místa spojení v položce pořadníku, namísto toho stačí uvádět jen vypočtenou minimální vzdálenost, která může (ale nemusí) nahradit minimální vzdálenost v tabulce.

Poslední navržený algoritmus vyžaduje dvě pomocné struktury.

*pořadník*: seznam dvojic naplánovaných průchodů spojeními (koncové město spojení, a nabízená minimální délka spojení). Do seznamu/pořadníku přidáváme na konec a vyjímáme (zpracování+smazání) na začátku. Seznam do něhož přidáváme prvky na konci a vyjímáme je postupně na druhé straně (= počátku) se běžně označuje jako **fronta**).

*tabulka*: záznam o minimální vzdálenosti pro každé město. Lze použít seznam dvojic (jméno města, minimální vzdálenost), ale to je dost nepohodlné v případě změny minimální vzdálenosti. V tomto případě by bylo nutné původni dvojice odstraňovat. Naštěstí Python podporuje velké množství různých specializovaných kolekcí, z nichž si vybere téměř každý.

#### Slovník s nastavenou implicitní hodnotou

Nám se bude hodit kolekce `collections.defaultdict`, která efektivně ukládá dvojice (klíč, hodnota), včetně rychlého hledání hodnot podle klíče a jejich modifikace (v našem případě je klíčem město a hodnotou aktuální minimální vzdálenost). Navíc (na rozdíl od běžného slovníku) podporuje implicitní hodnotu pro klíče, které se ve slovníku nenachází. To se nám hodí, neboť náš algoritmus předpokládá, že město, které ještě nebylo dosaženo při hledání nejkratší cesty má vzdálenost nekonečnou (třída `float` umožňuje representovat i nekonečné hodnoty).

Slovník s implicitní hodnotou si vyzkoušíme na jednoduchém příkladě. Klíčem bude jméno studenta (řetězec), hodnotou informace o tom zda splnil podmínky zápočtu. Pokud jméno studenta ve slovníku není, pak se předpokládá, že zápočet nemá.

In [171]:
from collections import defaultdict

def implicit_value(): # tzv. tovární funkce produkující implicitní hodnoty
    return False

zapocty = defaultdict(implicit_value) # vytvoření slovníku (s registrací tovární funkce)
zapocty["Novák"] = True    # přidání dvojice (jméno, výsledek)
zapocty["Nejezchleba"] = False # přidání další dvojice

print( zapocty["Novák"]) # vypíše uloženou hodnotu pro klíč "Novák", což je True
print( zapocty["Nejezchleba"])  # pro jistotu vyzkoušíme i druhou uloženou hodnotu

print( zapocty["Snedlditetikasi"])  # vypíše hodnotu vracenou tovární metodou (= False)

True
False
False


**Úkol**: Vyzkoušejte, nahradit slovník `defaultdict` běžným slovníkem `dict`. (konstruktor běžného slovníku je vestavěný a nemusí mít žádné parametry).

In [174]:
from collections import defaultdict

def implicit_value(): # tzv. tovární funkce produkující implicitní hodnoty
    return False

zapocty = dict() # běžný slovník
zapocty["Novák"] = True    # přidání dvojice (jméno, výsledek)
zapocty["Nejezchleba"] = False # přidání další dvojice

print( zapocty["Novák"]) # chování je shhodné jako u slovníku s implicitní hodnotou
print( zapocty["Nejezchleba"])  # dtto

try:
    print( zapocty["Snedlditetikasi"])  # přístup k neexistující hodnotě však vede k výjimce
except Exception as e:
    print(e)

True
False
'Snedlditetikasi'


Nyní se vraťme k implementaci metody `distance` třídy `city`. Implemantace výše popsaného algoritmu je relativně přímočará. 

In [4]:
%%writefile cities.py
import math
from collections import defaultdict


def infinity_factory(): # tovární funkce pro vracení nekonečen
    return math.inf


class City:
    def __init__(self, name): # konstruktor
        self.name = name      # do objektu uložíme jméno města
        self.connections = [] # seznam spojení z města je zatím prázdný

    def add_connection(self, target, distance, bidi=True):
        self.connections.append((target, distance))  
        # přidáme dvojici (město, vzdálenost) do seznamu
        if bidi:  # je-li spojení obousměrné
            target.add_connection(self, distance, bidi=False) # vložíme i opačný směr

    def distance(self, target):
        # pořadník s plánem přesunu do startovního města (=self)
        waiting_list = [(self, 0)]
        # prázdná tabulka (nekonečné vzdálenosti)
        mindist = defaultdict(infinity_factory)
        while waiting_list: # dokud není pořadník prázdný
            # vyjme a vrátí první dvojici (další testovaný cíl, dosžitelná vzdálenost)
            goal, sum_distance = waiting_list.pop()
            # je-li dosažitelná vzdálenost menší než aktuální minimální
            if sum_distance < mindist[goal]:
                # nová minimální vzdálenost je rovna dosažitelné
                mindist[goal] = sum_distance
                # projdeme spojení do okolních měst (z města `goal`)
                for next_goal, next_distance in goal.connections:
                    # spočítáme dosažitelnou vzdálenost do okolních měst
                    next_sum_distance = sum_distance + next_distance
                    # je-li menší než aktuálná minimální
                    if next_sum_distance < mindist[next_goal]:
                        # pak přidáme spojení do pořadníku
                        waiting_list.append((next_goal, next_sum_distance))
        # a nakonec vrátíme minimální vzdálenost cílového města
        return mindist[target]

Overwriting cities.py


Po zapsání celého kódu do třídy `City` do souboru `cities.py`, můžeme do souboru připsat i malý testovací kód. 

In [6]:
from cities import City

if __name__ == "__main__":
    usti = City("Ústí nad Labem")
    teplice = City("Teplice")
    decin = City("Děčín")
    most = City("Most")
    lovosice = City("Lovosice")

    usti.add_connection(teplice, 21)
    usti.add_connection(decin, 24)
    decin.add_connection(teplice, 35)
    teplice.add_connection(most, 27)
    lovosice.add_connection(usti, 21)
    lovosice.add_connection(teplice, 25)
    lovosice.add_connection(most, 36)

    print(decin.distance(lovosice))

45


Podmínka na začátku testovacího kódu je typickým pythonským ideomem.  Pokud je soubor přímo vykonáván (tj. je hlavním programem), pak standardní proměnná `__name__` obsahuje řetězec `__main__` (v Pythonu si opravdu užijete podtržítek). Proto se vykoná i náš testovací kód s vytvořením čtyř měst a definicí jejich přímých spojení. Nakonec program vypíše vypočtenou vzdálenost mezi Lovosicemi a Děčínem (je to 45km).

Pokud však soubor použijeme jako modul (tj. budeme ho importovat z jiného kódu), pak proměnná `__name__` obsahuje jméno modulu (tj. zde `city`) a testovací kód se neprovede (což je správně neboť import má jen zavést nové třídy, funkce apod. nikoliv provádět nějaký podivný kód s nic neříkajícím textovým výstupem).

#### Importování vlastní knihovny

Jak již bylo řečeno náš skript lze importovat do jiných skriptů resp. Jupyter notebooku, pomocí příkazu `import`

In [7]:
import cities

Ne vždy se to však podaří.
Důvodem je skutečnost, že Python hledá moduly jen v některých adresářích. V zásadě se jedná o tři skupiny adresářů:

1. adresáře obsahující standardní knihovny daného překladače Pythonu včetně nainstalovaných knihoven třetích stran 
2. (nepovinně) speciální podadresář obsažený v rámci vašeho domovského adresáře
3. aktuální pracovní adresář interpreteru (proto to funguje v našem Jupyter notebooku)

Seznam prohledávaných adresářů lze získat v proměnné `sys.path` (nutno importovat standardní modul `sys`).

In [8]:
import sys
sys.path

['/home/fiser/ownCloud2/jupyter/python-opora',
 '/home/fiser/apps/intelpython3/lib/python37.zip',
 '/home/fiser/apps/intelpython3/lib/python3.7',
 '/home/fiser/apps/intelpython3/lib/python3.7/lib-dynload',
 '',
 '/home/fiser/apps/intelpython3/lib/python3.7/site-packages',
 '/home/fiser/apps/intelpython3/lib/python3.7/site-packages/IPython/extensions',
 '/home/fiser/.ipython']

Modul lze tak zpřístupnit:
    
1. umístěním skriptu do některého z vypsaných adresářů (výpis se může lišit pro různé překladače a platformy). V případě Linuxu resp. Mac OS lze místo přesunu/kopírování využít umístění symbolického odkazu do příslušného adresáře.
2. přidáním adresáře v němž je skript obsažen do seznamu

První přístup je vhodný, když chceme modul využívat dlouhodobě. Druhý se hodí jen pro krátkodobé testování (seznam cest k modulům se obnovuje po každém novém spuštění Pythonu).

> **Úkol**: Vyzkoušejte import vyýše uvedeného modulu v *PyCharm* (zkuste více řešení, tj. umístění ve projektovém adresáři, umístení do standardní cesty, rozšíření `sys.path`)

> **Úkol:** Vytvořte modul(skript) definující třídu representující přímku pomocí obecné rovnice ve tvaru `ax + by + c = 0`. Konstruktor přímky očekává parametry `a`, `b` a `c` obecné rovnice.

> Metoda `normal` vrací normálový vektor přímky jako dvojici (*tuple*).

> Implementujte také metodu `p1.parallel_to(p2)`, která zjistí vrátí `True`, pokud jsou přímky representované objekty s proměnnými `p1` a `p2` rovnoběžné. Podobná metoda `p1.perpendicular_to(p2)` testuje kolmost přímek.

> Modul by měl mít jméno `geometry`, třída pak jméno `Line`. Vyzkoušejte modul implementovat a v notebooku ověřte jeho funkčnost. Pokud modul vytváříte v `Pycharm` použijte pro něj nový projekt (na jménu nezáleží, zvolte například také `geometry`).

Následující snímek obrazovky ukazuje `Pycharm` zobrazující obsah skriptu `geometry.py` (tj. modulu `geometry`).

![Pycharm s module geometry](geometry_script.png)

Všimněte si definice funkce `parallel_to`, která řeší i případ, kdy je jedna ze souřadnic normálového vektoru nulová. Speciální zpracování této situace je nezbytné, neboť jinak by mohlo dojít k dělení nulou (to vede k výjimce).

Modul také obsahuje testovací kód, který ověřuje vztahy mezi třemi přímkami (`x` je rovnoběžné z `y`, `x` je kolmé k `z`). V praxi je to však nedostatečné, neboť se nekontrolují přímky s obecnými polohami či s nulovými souřadnicemi normálových vektorů.

Modul `geometry` je ještě nutné otestovat:

```python
import sys
sys.path.append("/home/fiser/PycharmProjects/geometry")
```

```python
from geometry import Line

x = Line(2,5,6)
y = Line(3,2,0)

print(x.parallel_to(y))
print(y.perpendicular_to(x))
```

```python
x = Line(2, 0, 8)
y = Line(5, 0, -1)

print(x.parallel_to(y))
print(y.perpendicular_to(x))
```

Práce s vektory pomocí n-tic nebo seznamů (v úkolu byly pro normálové vektory využity dvojice), je dost komplikovaná a nepřehledná, neboť Python nepodporuje ani základní vektorové operace (sčítání, skalární součet, apod.). V tomto případě je vhodné využít knihovnu `NumPy`, která se na manipulaci s vektory a vicedimenzionálními poli zaměřuje.